In [1]:
import gdspy
import pandas as pd
import numpy as np
import sys
import math
import pattern_writer as pw

In [2]:
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9
sys.setrecursionlimit(10**6)

In [3]:
def posArrayGenerator(a, b, numA, numB, generator_function, placement_function):
    arr=[]
    new_x=0
    new_y=0
    for x in range(-numA,numA+1):
        for y in range(-numB,numB+1):
            print(x)
            print(y)
            vec=[(x*a[0]+y*b[0], x*a[1]+y*b[1])]
            geometry_cell= generator_function(placement_function(vec[0]))
            arr.append(gdspy.CellReference(geometry_cell, (vec[0][0], vec[0][1]), magnification=1))
    return arr

In [4]:
def recArrayGen(a,b,dim,pos,prevDir, generator_function, placement_function, arr=[]):
    [dimX, dimY] = dim
    pax=(a[0]+pos[0])<=dimX/2 and (a[0] + pos[0])>=-dimX/2
    pay=(a[1]+pos[1])<=dimY/2 and (a[1] + pos[1])>=-dimY/2
    pa=bool(pax*pay)
    nax=(-a[0]+pos[0])<=dimX/2 and (-a[0] + pos[0])>=-dimX/2
    nay=(-a[1]+pos[1])<=dimY/2 and (-a[1] + pos[1])>=-dimY/2
    na=bool(nax*nay)
    
    pbx=(b[0]+pos[0])<=dimX/2 and (b[0] + pos[0])>=-dimX/2
    pby=(b[1]+pos[1])<=dimY/2 and (b[1] + pos[1])>=-dimY/2
    pb=bool(pbx*pby)
    nbx=(-b[0]+pos[0])<=dimX/2 and (-b[0] + pos[0])>=-dimX/2
    nby=(-b[1]+pos[1])<=dimY/2 and (-b[1] + pos[1])>=-dimY/2
    nb=bool(nbx*nby)
#     pax=(a[0]+pos[0])<=dimX and (a[0] + pos[0])>=0
#     pay=(a[1]+pos[1])<=dimY and (a[1] + pos[1])>=0
#     pa=bool(pax*pay)
    
#     pbx=(b[0]+pos[0])<=dimX and (b[0] + pos[0])>=0
#     pby=(b[1]+pos[1])<=dimY and (b[1] + pos[1])>=0
#     pb=bool(pbx*pby)
#     print("pa")
#     print(pa)
#     print(na)
#     print(pb)
#     print(nb)
    #if prevDir!=(-1,0) and pa:
    if (pos[0], pos[1]) in prev_pos:
        return arr
    
    prev_pos.append((pos[0], pos[1]))
    geometry_cell= generator_function(placement_function((pos[0], pos[1])))
    arr.append(gdspy.CellReference(geometry_cell, (pos[0], pos[1]), magnification=1))
    #print(prev_pos)
    if pa:
        vec=(a[0] + pos[0], a[1]+pos[1])
#         geometry_cell= generator_function(placement_function((pos[0], pos[1])))
#         arr.append(gdspy.CellReference(geometry_cell, (pos[0], pos[1]), magnification=1))
        recArrayGen(a,b,dim, vec, (1,0), generator_function, placement_function, arr=arr)
    
    #if prevDir!=(1,0) and na:
    if na:
        vec=(-a[0] + pos[0], -a[1]+pos[1])
#         geometry_cell= generator_function(placement_function((pos[0], pos[1])))
#         arr.append(gdspy.CellReference(geometry_cell, (pos[0], pos[1]), magnification=1))
        recArrayGen(a,b,dim, vec, (-1,0), generator_function, placement_function, arr=arr)

    #if prevDir!=(0,-1) and pb:
    if pb:
        vec=(b[0] + pos[0], b[1]+pos[1])
#         geometry_cell= generator_function(placement_function((pos[0], pos[1])))
#         arr.append(gdspy.CellReference(geometry_cell, (pos[0], pos[1]), magnification=1))
        recArrayGen(a,b,dim, vec, (0,1), generator_function, placement_function, arr=arr)
    
    #if prevDir!=(0,1) and nb:
    if nb:
        vec=(-b[0] + pos[0], -b[1]+pos[1])
#         geometry_cell= generator_function(placement_function((pos[0], pos[1])))
#         arr.append(gdspy.CellReference(geometry_cell, (pos[0], pos[1]), magnification=1))
        recArrayGen(a,b,dim, vec, (0,-1), generator_function, placement_function, arr=arr)
   
    
    return arr
    

In [5]:
gdspy.current_library = gdspy.GdsLibrary()
geometry1 = gdspy.Cell("CIRCLE")
geometry1.add(gdspy.Round((0, 0), 72*nm, tolerance=0.001))
geometry2 = gdspy.Cell("OVAL")
geometry2.add(gdspy.Round((0, 0), [72*nm, 64*nm], tolerance=1e-4))
def p_f(point):
    return np.sqrt(point[0]**2 + point[1]**2)
def g_f(distance):
    if distance < 25:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
        return geometry1
    else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
        return geometry2
#y = layout_placement2(230*nm, 570*nm,70,4,6,g_f, p_f)
#y = posArrayGenerator([4,2], [5,3], 2, 2 ,g_f, p_f)
a=[1,0]
b=[.5,math.sqrt(3)/2]
pos=(0,0)
prevDir=(0,0)
prev_pos=[]
arr=[]
dim=[6000, 6000]
cells=recArrayGen(a,b,dim,pos,prevDir, g_f, p_f, arr)

In [6]:
gdspy.current_library = gdspy.GdsLibrary()

Lens = gdspy.Cell("LENS")
Lens.add(cells)
gdspy.current_library.add(Lens)
gdspy.write_gds('Testing37.gds')

In [7]:
len(cells)

241

In [8]:
#len(prev_pos)